In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
from transformers import BitsAndBytesConfig
import numpy as np
import evaluate

In [3]:
dataset = load_dataset("wnut_17")

In [4]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, is_fast=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

Loading checkpoint shards: 100%|██████████| 3/3 [00:14<00:00,  4.73s/it]


In [6]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [7]:
def format_ner_example(example):
    tokens = example['tokens']
    labels = example['ner_tags']
    label_names = dataset["train"].features["ner_tags"].feature.names

    text = " ".join(tokens)
    entity_output = ""
    for token, tag in zip(tokens, labels):
        if tag != 0:  # 'O' is 0
            entity_output += f"- {token}: {label_names[tag]}\n"
    if not entity_output:
        entity_output = "No named entities found."

    prompt = f"Extract named entities from the following text:\n\nText: {text}\n\nAnswer:\n{entity_output}"
    return {"text": prompt}

In [8]:
dataset = dataset.map(format_ner_example)
dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding="max_length", max_length=512), batched=True)


Map: 100%|██████████| 1287/1287 [00:00<00:00, 6724.82 examples/s]


In [10]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
seqeval = evaluate.load("seqeval")
example = dataset["train"][0]
labels = [label_list[i] for i in example[f"ner_tags"]]

In [11]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [12]:
training_args = TrainingArguments(
    output_dir="./llama3-ner",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir="./logs",
    logging_steps=10,
    num_train_epochs=3,
    save_steps=500,
    fp16=True,
    report_to="none",
)

In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [14]:
model = model.to('cuda')

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(1000)),
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\rebal\AppData\Local\Temp\ipykernel_51660\1750803398.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [16]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\rebal\ner-tuning\ner\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.738100
20,3.058600
30,2.146100
40,1.975800
50,1.875200
60,1.912700
70,2.003000
80,1.910300
90,1.814400
100,1.880700


TrainOutput(global_step=375, training_loss=1.787155918121338, metrics={'train_runtime': 4790.6167, 'train_samples_per_second': 0.626, 'train_steps_per_second': 0.078, 'total_flos': 6.557653794816e+16, 'train_loss': 1.787155918121338, 'epoch': 3.0})

In [17]:
trainer.save_model()